In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import re

In [97]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

# Data Preprocessing

In [98]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [99]:
fake.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [100]:
fake['label'] = 0

In [101]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [102]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [103]:
true.isna().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [104]:
true['label'] = 1
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [105]:
print(fake.shape)
print(true.shape)

(23481, 5)
(21417, 5)


In [106]:
23481+21417

44898

In [107]:
df=pd.concat([fake, true], axis=0)

In [108]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [109]:
df.shape

(44898, 5)

In [110]:
df.duplicated().sum()

209

In [111]:
df=df.drop_duplicates()

In [112]:
df.duplicated().sum()

0

# Feature extraction

In [113]:
df = df[['text', 'label']]
df.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [114]:
df.shape

(44689, 2)

# Stemming ( Removing stopwords and unnecessary words from the text column )

In [115]:
port_stem = PorterStemmer()

In [116]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [93]:
def noise_text(text):
    te = re.sub('[^a-zA-Z]',' ',text)
    te = te.lower()
    te = te.split()
    te = [port_stem.stem(word) for word in te if not word in stopwords.words('english')]
    te = ' '.join(te)
    return te

In [117]:
noise_text('My name is @@@ Shaniyaz %$%$^')

'name shaniyaz'

In [118]:
df.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [120]:
df['text']=df['text'].apply(noise_text)

In [70]:
df=pickle.load(open('stem_data.pkl','rb'))
df=pd.DataFrame(df)

In [71]:
df.head()

,text,label
0,washington reuter head conserv republican fact...,1
1,washington reuter transgend peopl allow first ...,1
2,washington reuter special counsel investig lin...,1
3,washington reuter trump campaign advis georg p...,1
4,seattl washington reuter presid donald trump c...,1


In [72]:
df.shape

(23481, 2)

In [4]:
vect=TfidfVectorizer()

In [16]:
x=df['text']
y=df.iloc[:,-1]
x

0        washington reuter head conserv republican fact...
1        washington reuter transgend peopl allow first ...
2        washington reuter special counsel investig lin...
3        washington reuter trump campaign advis georg p...
4        seattl washington reuter presid donald trump c...
                               ...                        
23479    st centuri wire say al jazeera america go hist...
23480    st centuri wire say wire predict new year look...
9942     washington reuter u hous repres speaker paul r...
11446    zurich reuter referendum switzerland clarifi c...
14925    istanbul reuter turkey russia agre focu polit ...
Name: text, Length: 23481, dtype: object

In [ ]:
#pickle.dump(df.to_dict(),open('stem_data.pkl','wb'))

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [18]:
x_train_vector = vect.fit_transform(x_train).toarray()
x_test_vector = vect.transform(x_test).toarray()

In [ ]:
x_train_vector

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
lr = LogisticRegression()

In [22]:
lr.fit(x_train_vector,y_train)

LogisticRegression()

In [36]:
def Fake_news(news):
    te = re.sub('[^a-zA-Z]',' ',news)
    te = te.lower()
    te = te.split()
    te = [port_stem.stem(word) for word in te if not word in stopwords.words('english')]
    te = ' '.join(te)
    te=[te]
    te = vect.transform(te).toarray()
    te=lr.predict(te)
    return te

In [69]:
pre = Fake_news('Trump Spent The Night Retweeting A White Supremacist Convicted Criminal,"While most of us were sleeping, Donald Trump spent his night retweeting a white supremacist who was convicted in the UK for harassing a Muslim woman in front of her two young children. Jayda Fransen was arrested in October during a neo-Nazi rally for violating terms of her bail. Fransen, Deputy Director of Britain First is thrilled to have been retweeted three times by the so-called president of the United States. Trump even retweeted a video of a boy on crutches being murdered, just in case you haven t had enough gore in your life.The former reality show star s retweets even stunned Paul Joseph Watson, Infowars editor-at-large who suggested the move was  not great optics. Yeah, someone might want to tell whoever is running Trump Twitter account this morning that retweeting Britain First is not great optics.   Paul Joseph Watson (@PrisonPlanet) November 29, 2017Even Infowars thinks Trump went too far. Let that sink in for a moment.Here s a screen capture of the snuff film Trump retweeted.Fransen was super excited that Trump shared her anti-Muslim videos to his 44 million followers and went all caps in her tweet. THE PRESIDENT OF THE UNITED STATES, DONALD TRUMP, HAS RETWEETED THREE OF DEPUTY LEADER JAYDA FRANSEN S TWITTER VIDEOS!  Fransen tweeted.  DONALD TRUMP HIMSELF HAS RETWEETED THESE VIDEOS AND HAS AROUND 44 MILLION FOLLOWERS! GOD BLESS YOU TRUMP! GOD BLESS AMERICA! THE PRESIDENT OF THE UNITED STATES, DONALD TRUMP, HAS RETWEETED THREE OF DEPUTY LEADER JAYDA FRANSEN TWITTER VIDEOS! DONALD TRUMP HIMSELF HAS RETWEETED THESE VIDEOS AND HAS AROUND 44 MILLION FOLLOWERS! GOD BLESS YOU TRUMP! GOD BLESS AMERICA! OCS @JaydaBF @realDonaldTrump pic.twitter.com/BiQfQkTra9  Jayda Fransen (@JaydaBF) November 29, 2017Trump cult member Piers Morgan was shocked by the retweets, too.  Good morning, Mr President @realDonaldTrump   what the hell are you doing retweeting a bunch of unverified videos by Britain First, a bunch of disgustingly racist far-right extremists?  he tweeted.  Please STOP this madness & undo your retweets. Good morning, Mr President @realDonaldTrump   what the hell are you doing retweeting a bunch of unverified videos by Britain First, a bunch of disgustingly racist far-right extremists? Please STOP this madness & undo your retweets.  Piers Morgan (@piersmorgan) November 29, 2017The video of the man being pushed off of a rooftop was filmed in Egypt during the 2013 overthrow of president Mohamed Morsi and the man complicit of the crime was hanged after being found guilty, according to BuzzFeed News.When British Labour Party politician Jo Cox was murdered by a far-right terrorist, he shouted,  This is for Britain ,  keep Britain independent , and  Britain first  as he attacked her.Cox s husband responded to Trump s retweets, writing,  Trump has legitimised the far right in his own country, now he s trying to do it in ours. Spreading hatred has consequences & the President should be ashamed of himself. Trump has legitimised the far right in his own country, now he s trying to do it in ours. Spreading hatred has consequences & the President should be ashamed of himself.  Brendan Cox (@MrBrendanCox) November 29, 2017Trump just promoted a woman who was convicted of a hateful act in Britain, much to the distress of a widowed man whose wife s murder shocked the world. Either Trump doesn t know how to Google or he s a hateful bigot   or both. You choose.Photo by Zach Gibson   Pool/Getty Images.",News,"November 29, 2017')
print(pre)

[1]


In [40]:
lr.score(x_test_vector,y_test)*100

98.38194592292953

In [ ]:
import gradio
gradio.Interface()

In [123]:
df2=pickle.load(open('stem_data.pkl','rb'))
pd.DataFrame(df2)

,text,label
0,washington reuter head conserv republican fact...,1
1,washington reuter transgend peopl allow first ...,1
2,washington reuter special counsel investig lin...,1
3,washington reuter trump campaign advis georg p...,1
4,seattl washington reuter presid donald trump c...,1
...,...,...
23479,st centuri wire say al jazeera america go hist...,0
23480,st centuri wire say wire predict new year look...,0
9942,washington reuter u hous repres speaker paul r...,1
11446,zurich reuter referendum switzerland clarifi c...,1
